In [11]:
import pandas as pd
import sqlite3

In [12]:
connect_db = sqlite3.connect('checking-logs.sqlite')


In [13]:
query = '''
SELECT time,
       avg(d) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(test.first_commit_ts) - 
                    JulianDay(datetime(deadlines.deadlines, 'unixepoch'))
                    ) * 24 as Integer
                  ) as d,
             CASE 
                WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
                ELSE 'after' 
            END AS time
       FROM test left join deadlines on test.labname = deadlines.labs
       WHERE labname <> 'project1'
      )
WHERE uid in (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
                                ELSE 'after' 
                           END as time
                    FROM test
                    LEFT JOIN deadlines ON test.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
               GROUP BY uid
               HAVING COUNT(DISTINCT time)=2
               )
GROUP BY time
'''
test_results = pd.io.sql.read_sql(query, connect_db)
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [14]:
connect_db.close()

Гипотеза не подтвердилась